
# Urban Population Growth and CO₂ Emissions: Conflicting Perspectives  
### Data Story Project – M2 Draft  
**Group B5 – #5**  
Jens Menkveld (15768643) • Paul Elsinghorst (15002608) • Benjamin Didic (14986876) • Jonne Bijman (15801179)  
*Draft created: June 16, 2025*



## Table of Contents  
1. [Introduction](#Introduction)  
2. [Perspectives & Arguments](#Perspectives)  
3. [Datasets & Pre‑processing](#Datasets)  
4. [Visualization Drafts](#Visualizations)  
5. [Structure & Next Steps](#NextSteps)  



## 1  Introduction <a id="Introduction"></a>  
Cities are magnets for people and opportunity. In 1990 roughly 2.3 billion people lived in urban areas; by 2020 that figure had grown to almost 4.4 billion.  
Many economists see this rapid urbanisation as the engine of productivity and innovation, lifting millions out of poverty. Environmental scientists, however, warn that the same concentration of people and activity intensifies CO₂ emissions, traffic congestion and energy demand, jeopardising climate goals.  

This data‑story explores that tension through the lenses of urban population growth and carbon‑intensive development, using publicly available global datasets.  
We aim to answer:  

*“Can we reap the economic benefits of big cities without locking‑in unsustainable levels of CO₂?”*  
And look at examples of this



## 2  Perspectives & Arguments <a id="Perspectives"></a>  

| Perspective | Key Arguments | Planned Visualisations |
|-------------|---------------|------------------------|
| **1 Economic Upside**: Growing cities fuel development | • Larger labour pools raise productivity<br>• Urban density fosters innovation clusters<br>• Consumer concentration stimulates markets | **V1** Line chart — Urban population trends (1990‑2020)<br>**V3** Scatter — Urban pop vs GDP<br>*(links Perspective 1 to V1 & V3)* |
| **2 Environmental Downside**: Rapid urbanisation drives emissions | • Cities account for >70 % of energy‑related CO₂<br>• Transport, heating & industry scale with population density<br>• High‐growth megacities show steep CO₂ per‑capita trajectories | **V2** Line chart — CO₂ per capita trends<br>**V4** Choropleth — CO₂ / capita latest year<br>**V5** Dual‑axis — Urban pop vs CO₂ for China<br>*(links Perspective 2 to V2, V4, V5)* |



## 3  Datasets & Pre‑processing <a id="Datasets"></a>  

Dataset 1 – World Development Indicators (WDI)  
*Indicator:* `SP.URB.TOTL` – Total urban population (1960‑2023)  
Source: World Bank Databank  


Dataset 2 – OWID CO₂ & Greenhouse Gas Emissions  
*Variable of interest:* `co2_per_capita` (t CO₂ per person)  
Source: Our World in Data (Global Carbon Project, etc.)

*Reproducibility:* The notebook downloads the latest CSVs directly from the providers.  


In [12]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np

pd.options.display.float_format = '{:,.2f}'.format



In [13]:

# -----------------------------
# 1. Load & tidy Urban Population
# -----------------------------
# Download once; cached locally afterwards
wdi_url = "https://api.worldbank.org/v2/en/indicator/SP.URB.TOTL?downloadformat=csv"
wdi_zip = "wdi_urban.zip"
urban_csv = None

try:
    import os, zipfile, requests, io, textwrap, warnings, tempfile
    if not os.path.exists(wdi_zip):
        print("Downloading World Bank data…")
        r = requests.get(wdi_url)
        with open(wdi_zip, "wb") as f:
            f.write(r.content)

    with zipfile.ZipFile(wdi_zip) as z:
        # The actual CSV has a long file‑name; pick the first one that starts with 'API_'
        for name in z.namelist():
            if name.startswith("API_") and name.endswith(".csv"):
                urban_csv = name
                break
        urban_pop = pd.read_csv(z.open(urban_csv), skiprows=4)
except Exception as e:
    warnings.warn(f"Automatic download failed: {e}\nFalling back to local stub…")
    # Fallback stub if offline
    urban_pop = pd.read_csv("data/urban_population_stub.csv")
urban_pop.head(n=5)


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,Urban population,SP.URB.TOTL,"27,887.00","28,212.00","28,580.00","28,917.00","29,221.00","29,502.00",...,"46,961.00","47,075.00","47,278.00","47,554.00","47,449.00","47,244.00","47,272.00","47,511.00",NaN,NaN
1,Africa Eastern and Southern,AFE,Urban population,SP.URB.TOTL,"18,960,189.00","19,796,785.00","20,690,699.00","21,653,526.00","22,685,293.00","23,779,650.00",...,"217,677,754.00","226,557,045.00","236,107,161.00","245,939,557.00","256,139,740.00","266,650,602.00","277,426,261.00","288,380,305.00",NaN,NaN
2,Afghanistan,AFG,Urban population,SP.URB.TOTL,"759,034.00","800,151.00","844,139.00","890,912.00","940,801.00","993,966.00",...,"8,682,093.00","9,011,456.00","9,367,638.00","9,749,465.00","10,168,092.00","10,525,708.00","10,800,465.00","11,165,011.00",NaN,NaN
3,Africa Western and Central,AFW,Urban population,SP.URB.TOTL,"14,361,518.00","15,050,171.00","15,775,196.00","16,550,889.00","17,374,634.00","18,251,823.00",...,"195,289,605.00","203,221,168.00","211,219,592.00","219,276,647.00","227,465,935.00","235,827,038.00","244,365,364.00","253,228,069.00",NaN,NaN
4,Angola,AGO,Urban population,SP.URB.TOTL,"545,923.00","572,465.00","599,897.00","628,663.00","658,872.00","690,469.00",...,"18,720,648.00","19,603,967.00","20,504,018.00","21,425,222.00","22,353,719.00","23,295,577.00","24,260,684.00","25,242,775.00",NaN,NaN


In [14]:
# Keep only needed years 1990‑2023
cols_keep = ['Country Name', 'Country Code'] + [str(y) for y in range(1990, 2024)]
urban_pop = urban_pop[cols_keep]

# Convert wide ➜ long
urban_long = urban_pop.melt(id_vars=['Country Name', 'Country Code'],
                            var_name='Year', value_name='Urban_Pop')
urban_long['Year'] = urban_long['Year'].astype(int)
urban_long.dropna(subset=['Urban_Pop'], inplace=True)

# Preview
urban_long.head()


,Country Name,Country Code,Year,Urban_Pop
0,Aruba,ABW,1990,"31,577.00"
1,Africa Eastern and Southern,AFE,1990,"78,899,581.00"
2,Afghanistan,AFG,1990,"2,550,909.00"
3,Africa Western and Central,AFW,1990,"64,881,850.00"
4,Angola,AGO,1990,"4,318,495.00"


The data preprocessing begins by downloading the World Bank’s urban population dataset in ZIP format, which contains a CSV file with yearly population data for each country. The script extracts the relevant CSV file and reads it into a DataFrame, skipping the first four rows that contain metadata. If the download fails, it falls back to loading a local stub file.

Next, it filters the dataset to retain only the columns for country name, country code, and the years 1990 to 2023. This trims unnecessary metadata and unrelated years. The data is originally in wide format, with one column per year. Then it is reshaped into long format using the melt() function, resulting in one row per country-year combination. The ‘Year’ column is converted to integers, and rows with missing population values are removed. The result is a tidy dataset suitable for time-series analysis or visualization of urban population trends across countries. 

In [15]:
# -----------------------------
# 2. Load & tidy CO₂ per capita
# -----------------------------
owid_url = "https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv"
try:
    co2 = pd.read_csv(owid_url)
except Exception as e:
    import warnings
    warnings.warn(f"Download failed: {e}\nTrying local stub…")
    co2 = pd.read_csv("data/owid_co2_stub.csv")

vars_we_need = ['country', 'iso_code', 'year', 'co2_per_capita', 'population', 'gdp']
co2 = co2[vars_we_need]
co2.rename(columns={'country':'Country Name',
                    'iso_code':'Country Code',
                    'year':'Year'}, inplace=True)
co2 = co2.query("Year >= 1990")

# Preview
co2.head()


,Country Name,Country Code,Year,co2_per_capita,population,gdp
240,Afghanistan,AFG,1990,0.17,"12,045,664.00","13,065,984,000.00"
241,Afghanistan,AFG,1991,0.16,"12,238,879.00","12,047,362,048.00"
242,Afghanistan,AFG,1992,0.11,"13,278,982.00","12,677,539,840.00"
243,Afghanistan,AFG,1993,0.10,"14,943,174.00","9,834,582,016.00"
244,Afghanistan,AFG,1994,0.09,"16,250,799.00","7,919,856,640.00"


In [16]:

# Merge one‑to‑one on Country & Year for analyses
merged = pd.merge(urban_long, co2, on=['Country Name', 'Country Code', 'Year'], how='inner')

# Quick sanity check
merged.describe()[['Urban_Pop', 'co2_per_capita', 'population', 'gdp']]


,Urban_Pop,co2_per_capita,population,gdp
count,"6,086.00","6,011.00","6,086.00","4,717.00"
mean,"16,475,529.78",5.17,"33,485,093.68","498,586,553,475.69"
std,"59,350,531.00",8.36,"136,029,655.48","1,806,305,908,612.36"
min,"3,577.00",0.02,"8,821.00","257,172,000.00"
25%,"525,195.25",0.62,"1,166,395.25","18,819,405,824.00"
50%,"3,080,348.00",2.65,"6,169,678.00","57,735,684,096.00"
75%,"8,677,608.25",7.12,"18,640,599.25","267,081,351,168.00"
max,"910,895,447.00",364.69,"1,438,069,597.00","26,966,017,179,648.00"



## 4  Visualization Drafts <a id="Visualizations"></a>  
Below are six visual components. The first four are close to finished;  
V5–V6 are rough prototypes or code‑skeletons that we will refine after peer feedback.


In [31]:
#1e en 2e vis misschien bij elkaar toevoegen
focus_countries = ['China', 'India', 'United States', 'Germany', 'Nigeria']
fig1 = px.line(urban_long[urban_long['Country Name'].isin(focus_countries)],
               x='Year', y='Urban_Pop', color='Country Name',
               title='Urban Population Growth, 1990‑2023')
fig1.update_yaxes(title='Urban population')
fig1.show()


In [26]:

fig2 = px.line(co2[co2['Country Name'].isin(focus_countries)],
               x='Year', y='co2_per_capita', color='Country Name',
               title='CO₂ Emissions per Capita, 1990‑2023',
               labels={'co2_per_capita':'t CO₂ per person'})
fig2.show()


In [24]:
#deze misschien per continent/ development factor op kleur sorteren + minimale grootte datapoints, legende nodig?
latest = merged[merged['Year'] == 2020]
fig3 = px.scatter(latest, x='Urban_Pop', y='co2_per_capita',
                  size='population', hover_name='Country Name',
                  title='Urban Population vs CO₂ per Capita (2020)',
                  labels={'Urban_Pop':'Urban population',
                          'co2_per_capita':'t CO₂ per person'})
fig3.update_xaxes(type='log')
fig3.show()


In [27]:
# Hier slider van jaren toevoegen + hoeveel c02 per capita? is het in ton?
fig4 = px.choropleth(latest,
                    locations='Country Code',
                    color='co2_per_capita',
                    hover_name='Country Name',
                    color_continuous_scale='Reds',
                    title='CO₂ per Capita by Country (2020)')
fig4.show()


In [33]:
#hier misschien tussen een set landen kunnen kiezen, correlatie berekenen
cn = merged[merged['Country Name'] == 'China']
fig5 = go.Figure()
fig5.add_trace(go.Bar(x=cn['Year'], y=cn['Urban_Pop'],
                      name='Urban Population', opacity=0.6, yaxis='y1'))
fig5.add_trace(go.Scatter(x=cn['Year'], y=cn['co2_per_capita'],
                          name='CO₂ per Capita', yaxis='y2'))
fig5.update_layout(
    title='China: Urban Population (bars) vs CO₂ per Capita (line)',
    yaxis=dict(title='Urban Population'),
    yaxis2=dict(title='CO₂ per Capita (t)', overlaying='y', side='right')
)
fig5.show()


In [32]:
#schaal miss logaritmisch + grafiek hieronder met gdp per capita ook veradering
urb90 = urban_long.query("Year == 1990")[['Country Code', 'Urban_Pop']].rename(columns={'Urban_Pop':'Pop1990'})
urb20 = urban_long.query("Year == 2020")[['Country Code', 'Urban_Pop']].rename(columns={'Urban_Pop':'Pop2020'})
chg = pd.merge(urb90, urb20, on='Country Code')
chg['pct_change'] = (chg['Pop2020'] - chg['Pop1990']) / chg['Pop1990'] * 100

# draft map
fig6 = px.choropleth(chg,
                     locations='Country Code',
                     color='pct_change',
                     color_continuous_scale='Viridis',
                     title='Urban Population Growth 1990‑2020 (% change)')
fig6.show()



## 5  Structure & Next Steps <a id="NextSteps"></a>  

* What’s complete so far  
  * **Introduction** for a general audience  
  * Two perspectives & mapped arguments  
  * Cleaned datasets stored in `data/` or downloaded programmatically  
  * **4/6** visualisations ~90 % ready  
  * Draft code & skeleton for remaining visuals  

* Planned refinements  
  1. Add GDP variables to strengthen economic perspective (V3 enhancement)  
  2. Improve colour palettes & accessibility annotations  
  3. Write narrative captions linking each visual to an argument  
  4. Implement guided scrolling (“scrollytelling”) layout in final deployment  
  5. Peer‑review code for reproducibility and performance issues
  6. gdp per capita grafieken* urbanisatie
  7. argumenten duidelijk verwerken
